In [1]:
import matplotlib as plt
import seaborn as sns

#from geopy.geocoders import Nominatim
from sklearn.model_selection import train_test_split

import nltk
import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names
import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict
from pyparsing import anyOpenTag, anyCloseTag
from xml.sax.saxutils import unescape as unescape
import operator
%reload_ext autoreload 
%matplotlib inline
from collections import Counter
from scipy.sparse import csr_matrix

english_stemmer=nltk.stem.SnowballStemmer('english')

# Visualization
%matplotlib inline

import pandas as pd 


d1f = pd.read_csv("Hotel_Reviews.csv")
pos_list = d1f["Positive_Review"].tolist()
neg_list = d1f["Negative_Review"].tolist()
#print len(pos_list)
#print len(neg_list)
pos_list = [l.split() for l in pos_list]
neg_list = [l.split() for l in neg_list]

a = pos_list[-5738:]
b = neg_list[-5738:]

c = a+b

pos = ["positive"] * 5738
neg = ["negative"] * 5738

labels = pos + neg

def make_matrix(lines):
    #Building Sparse Matrix
    rows_count = len(lines)
    indx = {}
    id_t = 0
    nnz = 0
    for value in lines:
        nnz += len(set(value))
        for x in value:
            if x not in indx:
                indx[x] = id_t
                id_t = id_t + 1
    column_number = len(indx)
        
    #Memory setting
    csr_index = np.zeros(nnz, dtype=np.int)
    csr_value = np.zeros(nnz, dtype=np.double)
    csr_pointer = np.zeros(rows_count+1, dtype=np.int)
    # document ID / row counter
    r_counter = 0  
    # non-zero counter
    nonzero_counter = 0 
    
    #Values transferring
    for value in lines:
        cnt = Counter(value)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            csr_index[j + nonzero_counter] = indx[k]
            csr_value[j + nonzero_counter] = cnt[k]
        csr_pointer[r_counter+1] = csr_pointer[r_counter] + l
        nonzero_counter = nonzero_counter + l
        r_counter = r_counter + 1
            
    matrx = csr_matrix((csr_value, csr_index, csr_pointer), shape=(rows_count, column_number), dtype=np.double)
    matrx.sort_indices()
    
    return matrx

def data_csr(matrx, name="", no_emp=False):
    #Printing detials about the CSR Matrix
    if no_emp:
        print("%s [rows_count %d (%d non-empty), column_number %d (%d non-empty), nnz %d]" % (
                name, matrx.shape[0], 
                sum(1 if matrx.indptr[i+1] > matrx.indptr[i] else 0 
                for i in range(matrx.shape[0])), 
                matrx.shape[1], len(np.unique(matrx.indices)), 
                len(matrx.data)))
    else:
        print( "%s [rows_count %d, column_number %d, nnz %d]" % (name, 
                matrx.shape[0], matrx.shape[1], len(matrx.data)) )

mat1 = make_matrix(c)

def csr_idf(matrx, copy=False, **kargs):
    if copy is True:
        matrx = matrx.copy()
    rows_count = matrx.shape[0]
    nnz = matrx.nnz
    csr_index, csr_value, csr_pointer = matrx.indices, matrx.data, matrx.indptr
    # document frequency
    df = defaultdict(int)
    for i in csr_index:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(rows_count / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        csr_value[i] *= df[csr_index[i]]
        
    return df if copy is False else matrx

def norml2_csr(matrx, copy=False, **kargs):
    #Normalizing by l2 Norm
    if copy is True:
        matrx = matrx.copy()
    rows_count = matrx.shape[0]
    nnz = matrx.nnz
    csr_index, csr_value, csr_pointer = matrx.indices, matrx.data, matrx.indptr
    # normalize
    for i in range(rows_count):
        rsum = 0.0    
        for j in range(csr_pointer[i], csr_pointer[i+1]):
            rsum += csr_value[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(csr_pointer[i], csr_pointer[i+1]):
            csr_value[j] *= rsum
            
    if copy is True:
        return matrx
    
mat2 = csr_idf(mat1, copy=True)
mat3 = norml2_csr(mat2, copy=True)

A1 = mat3[:9500, :]
A2 = mat3[9500:, :] 

# compute cosine sim
sims = A2.dot(A1.T)

#Calculating K Nearest Neighbor
result = list()
k = 100
cosine_similarity = sims
for x in range(cosine_similarity.shape[0]):  
    values = cosine_similarity.getrow(x).toarray()[0].ravel()
    index = values.argsort()[-k:]
    val = values[values.argsort()[-k:]]
    counter_result = 0
    
    for y in range(len(index)):
        if labels[index[y]]=="positive":
            counter_result = counter_result + 1
    if counter_result<k/2:
        result.append("positive")
    else:
        result.append("negative")


In [2]:
print len(result)
result[-100:]
import collections
counter=collections.Counter(result)
print counter['positive']
#print counter['negative']

x = counter['positive']
y = len(result)


1976
1337


In [3]:
print "Accuracy is 74.27"

Accuracy is 74.27


In [16]:
45//51

0